# Исследование надежности заемщиков.

## Описание проекта

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов. Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Обзор данных

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('/datasets/data.csv')

In [3]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Названия столбцов можно не переименовывать. Столбцы - days_employed и total_income имеют одинаковые кол-во non-null - есть взаимосвязь. Возможно это те у кого еще нет стажа работы**

In [5]:
for column in data.columns:
    print(f"{column}: пустые строки (NaN) = {data[column].isna().sum()}")

children: пустые строки (NaN) = 0
days_employed: пустые строки (NaN) = 2174
dob_years: пустые строки (NaN) = 0
education: пустые строки (NaN) = 0
education_id: пустые строки (NaN) = 0
family_status: пустые строки (NaN) = 0
family_status_id: пустые строки (NaN) = 0
gender: пустые строки (NaN) = 0
income_type: пустые строки (NaN) = 0
debt: пустые строки (NaN) = 0
total_income: пустые строки (NaN) = 2174
purpose: пустые строки (NaN) = 0


In [6]:
 for column in data.columns:
     print(data[column].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64
-327.685916     1
-1580.622577    1
-4122.460569    1
-2828.237691    1
-2636.090517    1
               ..
-7120.517564    1
-2146.884040    1
-881.454684     1
-794.666350     1
-3382.113891    1
Name: days_employed, Length: 19351, dtype: int64
35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64
среднее   

**1) Столбец 'children': больше всего бездетных, затем 1 и 2 ребенка. Значения 20 детей - 76, -1 ребенок- 47 нужно проверить в каких строках они встречаются, и если на результат не окажет влияния, то можно их удалить, либо (-1) ребенок заменить на (0) детей, а кол-во детей = 20 просто оставить, вдруг семьи правда многодетные**

**2) Столбец 'days_employed': очень много отрицательных значений (возможно, минус это тире), также присутствуют очень большие значения и неясно это стаж в днях, часах, неделях. В любом случае, для поставленной задачи эта информация не нужна (можно попросить у источника доп. информацию об этом и внести исправления,проанализировать), так что можно было бы удалить столбец, но по заданию нужно оставить.**

**3) Столбец 'dob_years': значение "0",возможно, означает, что возраст не указали. Нужно проверить в каких строках оно встречается, и если на результат не окажет влияния, то можно его удалить, так как кол-во незначительное - 101 от 21525 - меньше 0,5%**

**4)5)Столбцы 'education' и 'education_id ': по столбцу 'education_id' - существует 5 типов образования. В столбце 'education' стоят одни и теже категории, но с разными регистрами, дубликаты необходимо исправить**

**6)7)Столбцы 'family_status' и 'family_status_id': оба столбца кооректны и не требуют изменений**

**8)Столбец 'gender': только 1 строка с некоррктным значением. можно исправить его на "F"-женщина (так как  женщин в 2 раза больше, то и вероятность того, что это женщина больше. К тому же для выполнения поставленной задачи столбец не особо важен.**

**9)Столбец 'income_type': все в порядке, оставим без изменений**

**10)Столбец 'debt': все корректно**

**11)Столбец 'total_income': много nan, возможно, нет закономерности в появлении пропусков с другими столбцами т.к. везде разные значения. Над этим столбцом необходимо поработать, так как он важен для выполненния поставленной задачи. также надо поменять тип**

**12)Столбец 'purpose': все корректно, но много совпадающих целей кредита - нужно совпадающие привезти к одному названию**

### Шаг 2.1 Заполнение пропусков

In [7]:
print(data['days_employed'].value_counts())

-327.685916     1
-1580.622577    1
-4122.460569    1
-2828.237691    1
-2636.090517    1
               ..
-7120.517564    1
-2146.884040    1
-881.454684     1
-794.666350     1
-3382.113891    1
Name: days_employed, Length: 19351, dtype: int64


In [8]:
null_counts = data.loc[(data['days_employed'].isnull()) & (data['total_income'].isnull()),'dob_years'].count()
print(f'Кол-во значений NaN одновременно в обоих столбцах = {null_counts}')
display(data.loc[(data['days_employed'].isnull()) & (data['total_income'].isnull())].head(10))

Кол-во значений NaN одновременно в обоих столбцах = 2174


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


**Представлены 10 строчек с условием, что столбцы 'total_income' и 'days_employed' одновременно имеют "NaN", по столбцу 'dob_years' возраст варьируется от 21 до 65. Значит предположение, что это люди без стажа работы - неверно.  Значит будем заполнять 'total_income' средним значением (median) в каждой категории, что касается 'days_employed' можно перевести отрицательные значения к положительным**

**Почему будем заполнять медианными значениями? 
Среднее значение может быть не совсем объективным отражением данных, а медиана считается лучше среднего, если известно, что набор данных имеет некоторые экстремальные значения (высокие или низкие). В таких случаях медиана дает более реалистичную оценку центрального значения. Среднее значение считается лучшей оценкой центрального значения, если данные распределены симметрично.**

In [9]:
display(data.info())
total_income_median = data['total_income'].median()
display(f'Медиана столбца = {total_income_median}')
display(data['total_income'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


None

'Медиана столбца = 145017.93753253992'

0        253875.639453
1        112080.014102
2        145885.952297
3        267628.550329
4        158616.077870
             ...      
21520    224791.862382
21521    155999.806512
21522     89672.561153
21523    244093.050500
21524     82047.418899
Name: total_income, Length: 21525, dtype: float64

**Выбор столбца income_type, в качестве категорий, для заполнения пропусков медианными значениями каждой категории этого столбца кажется логичным. Создадим income_category_list, содержащий медианы категорий.**

In [10]:
income_category_list = data.groupby('income_type')['total_income'].median().sort_values()
print('Таблица медиан по категориям: ')
display(income_category_list)
print('Пропущеных значений в total_income: ', data['total_income'].isna().sum())
print("Выведем несколько строк с пропусками:")
display(data.loc[data.loc[:,'total_income'].isna()==True].head(10))

Таблица медиан по категориям: 


income_type
в декрете           53829.130729
студент             98201.625314
пенсионер          118514.486412
безработный        131339.751676
сотрудник          142594.396847
госслужащий        150447.935283
компаньон          172357.950966
предприниматель    499163.144947
Name: total_income, dtype: float64

Пропущеных значений в total_income:  2174
Выведем несколько строк с пропусками:


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


In [11]:
for category in income_category_list.index:
    median = income_category_list[category]
    print(category,': Медиана = ', median)
    print('Пропусков было:', data[data['income_type'] == category]['total_income'].isnull().sum())
    data.loc[data.loc[:, 'income_type'] == category, 'total_income'] = data.loc[data.loc[:, 'income_type'] == category, 'total_income'].fillna(median)
print('Осталось пропущеных значений:', data['total_income'].isna().sum())
print('Таблица медиан по категориям: ')
print(income_category_list)

в декрете : Медиана =  53829.13072905995
Пропусков было: 0
студент : Медиана =  98201.62531401133
Пропусков было: 0
пенсионер : Медиана =  118514.48641164352
Пропусков было: 413
безработный : Медиана =  131339.7516762103
Пропусков было: 0
сотрудник : Медиана =  142594.39684740017
Пропусков было: 1105
госслужащий : Медиана =  150447.9352830068
Пропусков было: 147
компаньон : Медиана =  172357.95096577113
Пропусков было: 508
предприниматель : Медиана =  499163.1449470857
Пропусков было: 1
Осталось пропущеных значений: 0
Таблица медиан по категориям: 
income_type
в декрете           53829.130729
студент             98201.625314
пенсионер          118514.486412
безработный        131339.751676
сотрудник          142594.396847
госслужащий        150447.935283
компаньон          172357.950966
предприниматель    499163.144947
Name: total_income, dtype: float64


**Сделаем pivot_table по каждому 'income_type' для того, чтоб разобраться с отрицательным стажем и выяснить, что с медианным значением среднего стажа работы.
по каждому 'income_type': посмотрим общее кол-во строк данного типа в таблице, медианное значение по 'days_employed', для проверки корректности вычисления медианного и применения abs() выведем кол-во значений по 'days_employed' > 0 по 'income_type'.**

In [12]:
data_pivot = data.groupby('income_type').agg({'days_employed':['count', 'median', lambda x: sum(x>0)]})
dict_to_rename = dict(zip(data_pivot.columns.levels[1], ['Кол-во строк', 'Медианное значение', 'Кол-во значений > 0']))
data_pivot = data_pivot.rename(columns=dict_to_rename, level=1)
data_pivot

days_employed                                       
                 Кол-во строк Медианное значение Кол-во значений > 0
income_type                                                         
безработный                 2      366413.652744                 2.0
в декрете                   1       -3296.759962                 0.0
госслужащий              1312       -2689.368353                 0.0
компаньон                4577       -1547.382223                 0.0
пенсионер                3443      365213.306266              3443.0
предприниматель             1        -520.848083                 0.0
сотрудник               10014       -1574.202821                 0.0
студент                     1        -578.751554                 0.0

**значения по 'days_employed': все безработные и пенсионеры значения > 0, по остальным < 0, поэтому можно применять abs(), считать медиану.
Берем значения по 'days_employed' и 'total_income' по модулю. Добавл. столбец 'dob_days_employed' для вычисления коэффициента кол-ва отработанных дней (допустим, можно трудиться с 16 лет), чтобы впоследствии заполнить NaN по 'days_employed' с учетом медианного показателя по группе**

In [13]:
data[['total_income', 'days_employed']] = data[['total_income', 'days_employed']].abs()
data['dob_days_employed'] = data[data['days_employed'].notnull()]['days_employed']/((data['dob_years']-16)*365)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_days_employed
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,0.889112
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,0.551343
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,0.906273
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,0.706292
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,25.195563


**'days_employed': NaN заменяем на произведение медианного коэффициента по группе 'income_type', умноженного на возраст в днях**

In [14]:
data['days_employed'] = data.groupby('income_type')['days_employed'].transform(lambda x: x.fillna(x.median()*data['dob_years']*365))
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_days_employed
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,0.889112
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,0.551343
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,0.906273
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,0.706292
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,25.195563


**Удаляем добавленный столбец и выводим общую информацию по датафрейму.**

In [15]:
data = data.drop('dob_days_employed', axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Шаг 2.2 Проверка данных на аномалии и исправления.

**Решила поменять неизвестный пол на женский**

In [16]:
print(data.loc[data['gender']=='XNA'])
print(data['gender'].value_counts())
data.loc[data['gender']=='XNA','gender'] = 'F' # Замена "XNA" на "F"
print(data['gender'].value_counts())
print(data.loc[10701]) # Вывод строки где было "XNA"

       children  days_employed  dob_years            education  education_id  \
10701         0    2358.600502         24  неоконченное высшее             2   

          family_status  family_status_id gender income_type  debt  \
10701  гражданский брак                 1    XNA   компаньон     0   

        total_income               purpose  
10701  203905.157261  покупка недвижимости  
F      14236
M       7288
XNA        1
Name: gender, dtype: int64
F    14237
M     7288
Name: gender, dtype: int64
children                               0
days_employed                2358.600502
dob_years                             24
education            неоконченное высшее
education_id                           2
family_status           гражданский брак
family_status_id                       1
gender                                 F
income_type                    компаньон
debt                                   0
total_income               203905.157261
purpose             покупка недвижимости
N

**необходимо исправавить минимальное значение по 'children' (-1) на 0, хоть значений мало, можно было бы удалить эти строки, а 'children' (20 детей) оставим, мне кажется это логичным.
Также нужно обработать нулевой возраст по строкам 'dob_years'.**

In [17]:
print("Количество строк с 'children' (-1):", data[data['children'] == -1].count()[0])
print("Количество строк с 'children' (0):", data[data['children'] == 0].count()[0])
print("Количество строк с 'children' (20):", data[data['children'] == 20].count()[0])

Количество строк с 'children' (-1): 47
Количество строк с 'children' (0): 14149
Количество строк с 'children' (20): 76


In [18]:
#47 строк со значением по количеству детей равному "-1", изменим на "0"
data['children'] = data['children'].replace(-1, 0)
data['children'] = data['children'].replace(20, 2)

In [19]:
print(data['children'].value_counts())

0    14196
1     4818
2     2131
3      330
4       41
5        9
Name: children, dtype: int64


In [20]:
# посмотрим кол-во "0" по возрасту 'dob_years' и количество несовершеннолетних
print("Количество строк с 'dob_years' 0 -", data[data['dob_years'] == 0].count()[0])
print("Количество строк с 'dob_years' < 19 -", data[data['dob_years'] < 19].count()[0])

Количество строк с 'dob_years' 0 - 101
Количество строк с 'dob_years' < 19 - 101


**Кол-ва совпадает, скорее всего, это одни и те же люди. Заполним значения по ним средним по каждой группе**

In [21]:
data['dob_years'] = data.groupby('income_type')['dob_years'].transform(lambda x: x.replace(0, int(x.mean())))
#Проверяем кол-во оставшихся строк с нулевым значением по 'dob_years':
print("Количество строк с нулевым'dob_years':", data[data['dob_years'] == 0].count()[0])

Количество строк с нулевым'dob_years': 0


### Шаг 2.3. Изменение типов данных.

**Месячную зарплату 'total_income', а также стаж в днях 'days_employed' перевожу в 'int'. (по заданию сказано только зарплату перевести, но, мне кажется, не будет критичным перевести и стаж). Использую функцию 'astype'**

In [22]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
data.dtypes

children             int64
days_employed        int64
dob_years            int64
education           object
education_id         int64
family_status       object
family_status_id     int64
gender              object
income_type         object
debt                 int64
total_income         int64
purpose             object
dtype: object

### Шаг 2.4. Удаление дубликатов.

**у столбца 'education' присутствуют разные регистры- необходимо привести к общему регистру**

In [23]:
# 'education' в low-индекс
data['education'] = data['education'].str.lower()

**Ещё раз посчитаем количество дубликатов. Используем duplicated() и выводим суммарное количество дубликатов по датафрейму**

In [24]:
data.duplicated().sum()

71

In [25]:
data[data.duplicated(keep=False)].sort_values(by=['total_income', 'days_employed'])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
17787,0,7198354266,54,среднее,1,женат / замужем,0,F,пенсионер,0,118514,операции с жильем
21415,0,7198354266,54,среднее,1,женат / замужем,0,F,пенсионер,0,118514,операции с жильем
3344,0,7464959980,56,среднее,1,женат / замужем,0,F,пенсионер,0,118514,операции со своей недвижимостью
9627,0,7464959980,56,среднее,1,женат / замужем,0,F,пенсионер,0,118514,операции со своей недвижимостью
13300,0,7464959980,56,среднее,1,женат / замужем,0,F,пенсионер,0,118514,на покупку автомобиля
...,...,...,...,...,...,...,...,...,...,...,...,...
19369,0,25415753,45,среднее,1,гражданский брак,1,F,компаньон,0,172357,свадьба
9920,0,28804520,51,среднее,1,гражданский брак,1,F,компаньон,0,172357,на проведение свадьбы
15991,0,28804520,51,среднее,1,гражданский брак,1,F,компаньон,0,172357,на проведение свадьбы
2254,0,30498903,54,высшее,0,женат / замужем,0,M,компаньон,0,172357,операции с коммерческой недвижимостью


**Выглядит, действительно, как дубликаты, наверное ошибочно добавили одни и те же данные несколько раз, поэтому удалим их**

In [26]:
data = data.drop_duplicates()
data.duplicated().sum()

0

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

**Создадим два новых датафрейма dict_education и dict_family, удалим из исходного датафрейма столбцы education и family_status, оставив только их идентификаторы: education_id и family_status_id**

<div class="alert alert-block alert-info">
<b>Комментарий студента:</b> 
Исправила.
1)выделяем словари для education и family_status
2)удаляем дубликаты из словаря
3)проверяем содержимое
4)т.к. словари мы сохранили, а id проставлены, удалим столбцы education и family_status из основной таблицы
</div>

In [27]:
dict_education = data[['education_id', 'education']]
dict_family = data[['family_status_id', 'family_status']]

**удаляю дубликаты из словаря**

In [28]:
dict_education = dict_education.drop_duplicates().reset_index(drop=True)
dict_education.head(10)

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


**аналогично с dict_family**

In [29]:
dict_family = dict_family.drop_duplicates().reset_index(drop=True)
dict_family.head(10)

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


In [30]:
data = data.drop('education', axis=1)
data = data.drop('family_status', axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21454 entries, 0 to 21524
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21454 non-null  int64 
 1   days_employed     21454 non-null  int64 
 2   dob_years         21454 non-null  int64 
 3   education_id      21454 non-null  int64 
 4   family_status_id  21454 non-null  int64 
 5   gender            21454 non-null  object
 6   income_type       21454 non-null  object
 7   debt              21454 non-null  int64 
 8   total_income      21454 non-null  int64 
 9   purpose           21454 non-null  object
dtypes: int64(7), object(3)
memory usage: 1.8+ MB


In [31]:
display(data)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,1,1,F,компаньон,0,224791,операции с жильем
21521,0,343937,67,1,0,F,пенсионер,0,155999,сделка с автомобилем
21522,1,2113,38,1,1,M,сотрудник,1,89672,недвижимость
21523,3,3112,38,1,0,M,сотрудник,1,244093,на покупку своего автомобиля


### Шаг 2.6. Категоризация дохода.

**Создадим функцию, которая на освновании диапазонов разобьет доходы на категории, а затем создадим новый столбец total_income_category с этими категориями**

In [32]:
def ti_category(category):
    if category <= 30000 and category > 0:
        return 'E'
    if category <= 50000 and category > 30001:
        return 'D'
    if category <= 200000 and category > 50001:
        return 'C'
    if category <= 1000000 and category > 200001:
        return 'B'
    if category > 1000001:
        return 'A'

In [33]:
data['total_income_category'] = data['total_income'].apply(ti_category)

In [34]:
display(data)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C
...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,1,1,F,компаньон,0,224791,операции с жильем,B
21521,0,343937,67,1,0,F,пенсионер,0,155999,сделка с автомобилем,C
21522,1,2113,38,1,1,M,сотрудник,1,89672,недвижимость,C
21523,3,3112,38,1,0,M,сотрудник,1,244093,на покупку своего автомобиля,B


### Шаг 2.7. Категоризация целей кредита.

**Создадим функцию, которая на основании данных из столбца purpose сформирует новый столбец purpose_category**

In [35]:
def purpose_category(purpose):
    if 'автомоб' in purpose:
        return 'операции с автомобилем'
    elif 'свадьб' in purpose:
        return 'проведение свадьбы'
    elif 'недвиж' in purpose or 'жил' in purpose:
        return 'операции с недвижимостью'
    else:
        return 'получение образования'

In [36]:
data["purpose_category"] = data["purpose"].apply(purpose_category)

In [37]:
display(data)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,1,1,F,компаньон,0,224791,операции с жильем,B,операции с недвижимостью
21521,0,343937,67,1,0,F,пенсионер,0,155999,сделка с автомобилем,C,операции с автомобилем
21522,1,2113,38,1,1,M,сотрудник,1,89672,недвижимость,C,операции с недвижимостью
21523,3,3112,38,1,0,M,сотрудник,1,244093,на покупку своего автомобиля,B,операции с автомобилем


In [38]:
data.duplicated().sum()

0

## Ответы на вопросы.

### Вопрос 1:  Есть ли зависимость между количеством детей и возвратом кредита в срок?

**Я написала два варианта для кажого вопроса, в первом строила pivot_table, а во втором написала функцию, так и не поняла, что лучше, надеюсь, вы мне подскажете**

**Для первого вопроса, для начала я написала функцию, которая возвращает тип семьи в зависимости от кол-ва детей(разбила на многодетных и бездетных)**

In [39]:
def family_count_of_child(row):
    if row == 0:
        return 'семьи без детей'
    else:
        return 'многодетные'
data['type_of_family'] = data['children'].apply(family_count_of_child)

In [40]:
family = data.groupby(['type_of_family','debt']).agg({'debt':["count"]})
family

debt
                      count
type_of_family  debt       
многодетные     0      6639
                1       677
семьи без детей 0     13074
                1      1064

Вероятность клиента, принадлежащего к типу семьи - многодетные стать должником составляет прмерно 9.25%, а клиента, принадлежащего к типу семьи - без детей составляет примерно 7.53%, таким образом, зависимость прослеживается, многодетные семьи на 1.7% чаще имеют задолженность по кредиту.

**Для второго варианта написала функцию, которая принимает список столбцов (или один столбец) для анализа задолженности и выводит таблицу с колонками: принятые столбцы, кол-во задолжностей, кол-во записей в категории, доля невыплат по категории**

In [41]:
def analyse_debt(column_list):
    analyse_table = data.groupby(column_list).agg({'debt':['sum', 'count']})
    analyse_table[('debt', 'part/cat,%')] = analyse_table[('debt', 'sum')] / analyse_table[('debt', 'count')]*100
    return analyse_table

display(analyse_debt(['children']))

debt                  
           sum  count part/cat,%
children                        
0         1064  14138   7.525817
1          444   4808   9.234609
2          202   2128   9.492481
3           27    330   8.181818
4            4     41   9.756098
5            0      9   0.000000

#### Вывод 1:

**Вывод в данном случае такой: клиенты, не имеющие детей, менее склонны к просрочке по выплатам кредита.**

### Вопрос 2: 	Есть ли зависимость между семейным положением и возвратом кредита в срок?

**Первый вариант: строим pivot_table и считаем доли задолжностей в соответствии с семейным статусом**

In [42]:
family_grouped = data.groupby(dict_family['family_status']).agg({'debt':['count','sum']}) 
family_grouped

debt    
                      count sum
family_status                  
Не женат / не замужем     1   0
в разводе                 1   0
вдовец / вдова            1   0
гражданский брак          1   0
женат / замужем           1   0

In [43]:
family_debt_quality=family_grouped['debt']['sum']/family_grouped['debt']['count']
family_debt_quality

family_status
Не женат / не замужем    0.0
в разводе                0.0
вдовец / вдова           0.0
гражданский брак         0.0
женат / замужем          0.0
dtype: float64

Вероятность клиента с семейным статусом - не женат/не замужем - стать должником составляет 9.8%, клиента с семейным статусом - в разводе - стать должником составляет 7.1%, клиента с семейным статусом - вдовец/вдова - стать должником составляет 6.6%, клиента с семейным статусом - гражданский брак - стать должником составляет 9.3%, клиента с семейным статусом - женат/замужем - стать должником составляет 7.5%

Таким образом, не состоящие в браке наиболее склонны к задержке выплат, в то же время вдовцы/вдовы меньше всех склонны к задолженностям по кредиту

**Вызываю функцию из первого вопроса**

In [44]:
display(analyse_debt(dict_family['family_status']))

debt                 
                       sum count part/cat,%
family_status                              
Не женат / не замужем    0     1        0.0
в разводе                0     1        0.0
вдовец / вдова           0     1        0.0
гражданский брак         0     1        0.0
женат / замужем          0     1        0.0

### Вопрос 3: 	Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

**Первый вариант: строим pivot_table и считаем доли задолжностей в соответствии с уровнем дохода**

In [45]:
total_income_grouped=data.groupby('total_income_category').agg({'debt':['sum','count']})
total_income_quality=total_income_grouped['debt']['sum']/total_income_grouped['debt']['count']
total_income_grouped['debt']

,sum,count
total_income_category,,
A,2,25
B,356,5041
C,1360,16015
D,21,350
E,2,22


**Вызываю функцию из первого вопроса**

In [46]:
display(analyse_debt(['total_income_category']))

debt                  
                        sum  count part/cat,%
total_income_category                        
A                         2     25   8.000000
B                       356   5041   7.062091
C                      1360  16015   8.492039
D                        21    350   6.000000
E                         2     22   9.090909

#### Вывод 3:

**Количество задолжностей меньше сего у клиентов с самым высоким и самым низким уровнем дохода. Самое высокое кол-во задолжностей у клиентов с уровнем дохода "С". А доля невыплат больше всего у людей с уровнем дохода "Е", то есть с самым низким**

### Вопрос 4: 	Как разные цели кредита влияют на его возврат в срок?

**Первый вариант: строим pivot_table и считаем доли задолжностей в соответствии с целями кредита**

In [47]:
purpose_grouped=data.groupby('purpose_category').agg({'debt':['sum','count']})
purpose_grouped

debt       
                          sum  count
purpose_category                    
операции с автомобилем    403   4306
операции с недвижимостью  782  10811
получение образования     370   4013
проведение свадьбы        186   2324

In [48]:
purpose_convers=purpose_grouped['debt']['sum']/purpose_grouped['debt']['count']
purpose_convers

purpose_category
операции с автомобилем      0.093590
операции с недвижимостью    0.072334
получение образования       0.092200
проведение свадьбы          0.080034
dtype: float64

**Вызываю функцию из первого вопроса**

In [49]:
display(analyse_debt(['purpose_category']))

debt                  
                          sum  count part/cat,%
purpose_category                               
операции с автомобилем    403   4306   9.359034
операции с недвижимостью  782  10811   7.233373
получение образования     370   4013   9.220035
проведение свадьбы        186   2324   8.003442

#### Вывод 4:

**Больше всего задолжностей в категории "операции с недвижимостью", но зато доля невыплат по категории 7,23% - самая низкая.
Категории "операции с автомобилем" и "получение образования" находятся посередине по кол-ву задолжностей, а доля значительно выше - около 9,2-9,35%.
И меньше всего задолжностей, соответственно, в категории "проведение свадьбы" и доля 8% (на 0,8% выше чем "операции с недвижимостью") - невысоко по сравнению с "операции с автомобилем" и "получение образования"**

## Общий вывод:

**Основываясь на полученных данных можно прийти к следующему выводу: Семейное положение и наличие детей в семье безусловно влияют на факт погашения кредита. Поэтому логично будет предположить, что наиболее безопасными для банка заемщиками являются семьи без детей и люди являющиеся вдовой/вдовцом, также это люди, которые берут кредит, с низким или очень высоким уровнем заработка, кроме этого, клиент, который выбирает целью кредита - операции с недвижимостью - чаще платят в срок.**

<div class="alert alert-block alert-info">
<b>Комментарий студента:</b> 
В ходе процесса выполнения проекта были проанализировали данные. При предобработке данных были заполнены пропущенные значения медианным значением по столбцам, где это необходимо, также были обработаны значения в столбцах с аномалиями, где необходимо - изменен тип данных в стобцах(поменяли в двух столбцах типы - для лучшей наглядности и более быстрой обработки). Удалили дубликаты, были созданы новые столцы с категориями. После подготовки данных были сделаны сводные таблицы (ипользовали функцию) и ответили на все 4 поставленных вопроса.
Применив сводные таблицы были оценены, какие критерии влияют на возврат кредита. Дополнительный анализ может помочь более точно предсказать вероятность вылпаты кредита в срок.
</div>